# Chapter 11. Dataset

## DataSet이란

- DataSet은 `구조적 API의 기본 데이터 타입`이다.
- DataFrame은 스파크가 지원하는 다양한 언어에서 사용할 수 있으나, DataSet은 **자바 가상 머신을 사용하는 언어인 스칼라와 자바에서만** 사용할 수 있다. 
    - 스칼라에서는 스키마가 정의된 케이스 클래스 객체를 사용해 DataSet을 정의
    - 자바에서는 자바빈 객체를 사용해 DataSet을 정의
- DataFrame API를 사용할 때 String이나 Integer 데이터 타입의 객체를 생성하지는 않지만 **스파크는 Row 객체를 변환해 데이터를 처리한다.** 스칼라나 자바를 사용할 때 모든 DataFrame은 `Row 타입의 DataSet을 의미`한다. 
    - 데이타셋은 스파크 1.6에서 추가 되었다. **데이터의 타입체크**, **데이터 직렬화를 위한 인코더**, **카탈리스트 옵티마이저를 지원**하여 데이터 처리 속도를 더욱 증가시켰다.
    - 스파크 버전 2.0에서는 DataFrame을 Row 객체로 구성된 Dataset(즉, Dataset[Row])으로 구현했다. 또 DataFrame의 as 메서드를 사용해 DataFrame을 Dataset으로 변환할 수 있다.
    참고: https://thebook.io/006908/0152/

    ```scala
        # 스파크 소스 코드 중 org/apache/spark/sql/package.scala 파일에 DataFrame 선언
        type DataFrame = Dataset[Row]
    ```


- Dataset을 사용해 데이터셋의 각 로우를 구성하는 객체를 정의하며, 스파크 유경험자들은 타입형 API라고 부르기도 한다.
    
    - 데이타셋은 RDD와 유사하지만 객체를 직렬화 할때 자바의 기본 시리얼라이제이션이나 kyro를 사용하지 않고, 스파크의 인코더(Encoder) 를 이용하여 RDD 보다 속도가 빠르다.
    <img src="../../assets/presentations/week11/apach_spark_api.jpg" width="1000px" height="250px" title="catalyst_optimizer"/>
    참고: https://wikidocs.net/28556
    
    - Dataset의 핵심 아이디어는 “사용자에게 도메인 객체(domain object)에 대한 변환 연산을 손쉽게 표현할 수 있는 API를 지원함과 동시에, 스파크 SQL 실행 엔진의 빠른 성능과 높은 안정성을 제공하는 것”(https://goo.gl/PqKDu8)이다. 다시 말해 일반 자바 객체를 Dataset에 저장할 수 있고, 스파크 SQL의 텅스텐 엔진과 카탈리스트 최적화를 활용할 수 있다는 의미다.
    
    ** 도메인 객체: 도메인 객체는 문제 도메인에 딱 맞는 데이터를 정의한 커스텀 객체다. 데이터 전송 객체(Data Transfer Object, DTO)와 유사하다고 볼 수도 있겠지만, 단순히 데이터만 저장하는 DTO와 달리 도메인 객체는 데이터를 처리하는 로직도 포함하는 개념

    - Dataset은 스키마를 갖고 있다
        - (spark 1.5) DataFrame에 encoder를 추가한 것이 Dataset이라 할 수 있다.  
        - (spark 2.0) DataFrame은 특별한 encoder 없이 Dataset[Row]의 앨리어스로 쓸 수 있게 되었다.
        - 도메인별 특정 객체를 효과적으로 지원하기 위해 `인코더(encoder)`라 부르는 특수 개념이 필요하다. 
            
        ** 인코더: 인코더는 도메인별 특정 객체 T를 스파크의 내부 데이터 타입으로 매핑하는 시스템을 의미. 인코더는 런타임 환경에서 Person 객체를 **바이너리 구조로 직렬화하는 코드를 생성**하도록 스파크에 지시한다.

    - 예시) Person 클래스는 name과 int 두 개의 필드를 가지고 있다.
        - DataFrame이나 '표준' 구조적 API를 사용한다면 **Row 타입을 직렬화된 바이너리 구조로 변환**한다.

- DataFrame이나 '표준' 구조적 API를 사용한다면 Row 타입을 직력화된 바이너리 구조로 변환한다. 도메인 특화된 객체를 만들어 사용하려면 스칼라의 케이스 클래스 또는 자바의 자바빈 형태로 사용자 정의 데이터 타입을 정의해야한다. 스파크에서는 Row 타입 대신 사용자가 정의한 데이터 타입을 분산 방식으로 다룰 수 있다. 

- Dataset API를 사용한다면 스파크는 데이터셋에 접근할 때마다 Row 포맷이 아닌 사용자 정의 데이터 타입으로 변환합니다. 이 변환 작업은 느리긴 하지만 사용자에게 더 많은 유연성을 제공할 수 있다. 사용자 정의 데이터 타입을 사용하면 성능이 나빠지게 된다. 파이썬으로 만든 사용자 정의 함수와 비슷한 상황으로 볼 수도 있으나 자릿수가 다를 정도로 사용자 정의 함수가 훨씬 더 느리다.그 이유는 프로그래밍 언어를 전환하는 것이 사용자 정의 데이터 타입을 사용하는 것보다 훨씬 더 느리기 때문이다. 

## 11.1 Dataset을 사용할 시기
- `Dataset을 사용하면 성능이 떨어진다는데 사용할 필요가 있을까?`
    - DataFrame 기능만으로는 수행할 연산을 표현할 수 없는 경우
    - 성능 저하를 감수하더라도 타입 안정성(type-safe)을 가진 데이터 타입을 사용하고 싶은 경우

- 구조적 API를 사용해 표현할 수 없는 작업(예로는 복잡한 비즈니스 로직을 SQL이나 DataFrame 대신 단일 함수로 인코딩해야 하는 경우가 있음)의 경우 Dataset을 사용하는 것이 바람직하다. 
- Dataset API의 타입 안정성은 정확도와 방어적 코드를 중요하게 여길 때 필요하다. (예로는 두 문자열을 사용해 뺄셈 연산을 하는 것처럼 데이터 타입이 유효하지 않은 작업은 런타임이 아닌 컴파일 타임에 오류가 발생한다.)
    - Dataset API를 사용하면 잘못된 데이터로부터 애플리케이션을 보호할 수는 없지만 보다 우아하게 데이터를 제어하고 구조화할 수 있다. 
- 스칼라 창시자인 마틴 오더스키는 2015년 열린 스카프 서밋에서 "결국 Dataset을 사용하는 장점 중 하나는 로컬과 분산 환경의 워크로드에서 재사용할 수 있다는 겁니다."
    - 케이스 클래스로 구현된 데이터 타입을 사용해 모든 데이터와 트랜스포메이션을 정의하면 재사용할 수 있다. 올바른 클래스와 데이터 타입이 지정된 DataFrame을 로컬 디스크에 저장하면 다음 처리 과정에서 사용할 수 있어 더 쉽게 데이터를 다룰 수 있다. 

## 11.2 Dataset 생성
- Dataset을 생성하는 것은 수동 작업이므로 정의할 스키마를 미리 알고 있어야 한다. 

### 11.2.1 자바: Encoders
데이터 타입 클래스를 정의한 다음 DataFrame(Dataset<Row> 탕ㅂ의)에 지정해 인코딩할 수 있다.

```java
import org.apache.spark.sql.Encoders;

public Class Flight implements Serializable(
    String DEST_COUNTRY_NAME;
    String ORIGIN_COUNTRY_NAME;
    Long DEST_COUNTRY_NAME;
)

Dataset<Flight> flights = spark.read.parquet("/data/flight-data/parguet/2010-summary.parquet/")
.as(Encoders.bean(Flight.class));
```

### 11.2.2 스칼라: 케이스 클래스
스칼라에서는 Dataset을 생성하려면 스칼라 case class 구문을 사용해 데이터 타입을 정의해야 한다. 

- 케이스 클래스는 다음과 같은 특징을 가진 정규 클래스이다.
    - 불변성
    - 패턴 매칭으로 분해 가능
    - 참조값 대신 클래스 구조를 기반으로 비교
    - 사용하기 쉽고 다루기 편함

- 스칼라 문서는 케이스 클래스를 다음과 같이 설명한다. 
    - 불변성이므로 객체들이 언제 어디서 변경되었는지 추적할 필요가 없음
    - 값으로 비교하면 인스턴스를 마치 원시(primitive) 데이터 타입의 값처럼 비교한다. 그러므로 클래스 인스턴스가 값으로 비교되는지, 참조로 비교되는지 더는 불확실해하지 않아도 된다. 
    - 패턴 매칭은 로직 분기를 단순화해 버그를 줄이고 가독성을 좋게 만든다. 

```scala
case class Flight(
    DEST_COUNTRY_NAME: String,
    ORIGIN_COUNTRY_NAME: String,
    count: BigInt
)

val flightsDF = spark.read
    .parquet("/data/flight-data/parquet/2010-summary.parquet/")
val flights = flightsDF.as[Flight]
```
- Flight 데이터 타입은 스키마만 정의되었을 뿐 아무런 메서드도 정의되어 있지 않다. 데이터를 읽으면 DataFrame이 변환되고, as 메서드를 사용해 Flight 데이터 타입으로 변환한다.


### 참고
https://wikidocs.net/28556

- 데이터셋 초기화는 내부 데이터를 이용하는 방법과 외부 데이터를 이용하는 방법이 있다.

내부 데이터 이용한 초기화
``` scala
val seq =   Seq(
       ("David", 150),
       ("White", 200),
       ("Paul",  170)
  )
val peopleDS = seq.toDS()

scala> peopleDS.show()
+-----+---+
|   _1| _2|
+-----+---+
|David|150|
|White|200|
| Paul|170|
+-----+---+

scala> peopleDS.select("_1").show()
+-----+
|   _1|
+-----+
|David|
|White|
| Paul|
+-----+
```


케이스 클래스를 이용한 초기화
- 케이스 클래스를 이용한 내부 데이터 초기화
```scala
case class People(name: String, salary: Int)
val peopleSeq = Seq(
       People("David", 150),
       People("White", 200),
       People("Paul",  170)
  )
val peopleDS = peopleSeq.toDS()

scala> peopleDS.show()
+-----+------+
| name|salary|
+-----+------+
|David|   150|
|White|   200|
| Paul|   170|
+-----+------+


scala> peopleDS.select("salary").show()
+------+
|salary|
+------+
|   150|
|   200|
|   170|
+------+

```

RDD를 데이터셋으로 초기화
- RDD를 데이터셋으로 변환하기 위해서는 데이터 프레임으로 변경하고 데이터셋으로 변환하면 된다.
```scala
import org.apache.spark.sql._
import org.apache.spark.sql.types._

val peopleRDD = sc.textFile("/user/people.txt")
val peopleSchema = new StructType().add(StructField("name",   StringType, true)).add(StructField("age", IntegerType, true))
val sepPeopleRdd = peopleRDD.map(line => line.split(",")).map(x => Row(x(0), x(1).trim.toInt))
val peopleDF = spark.createDataFrame(sepPeopleRdd, peopleSchema)
peopleDF.show()

case class People(name: String, age: Long)
val peopleDS = peopleDF.as[People]
peopleDS.show()
```

데이터프레임을 데이터셋으로 초기화
- 데이터프레임을 데이터셋으로 변환하는 것은 데이터프레임에 정적데이터로 변경을 위해서 as를 이용해서 클래스를 지정해 주면 변환하면 된다. 스키마와 클래스의 이름이 동일하며 자동으로 바인딩 된다.
```scala
case class People(name: String, age: Long)
val peopleDF = spark.read.json("/user/people.json")
val peopleDS = peopleDF.as[People]

scala> peopleDS.show()
+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+
```


## 11.3 액션
- 데이터셋에도 데이터프레임에 collect, take, count 액션을 적용할 수 있다.
- 케이스클래스(case class)에 실제로 접근할 때 어떠한 데이터 타입도 필요하지 않다. 케이스클래스 속성명을 지칭하면 속성에 맞는 값과 데이터 타입 모두를 반환한다.
```
scala> flights.show(2)
  +-----------------+-------------------+-----+
  |DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
  +-----------------+-------------------+-----+
  |    United States|            Romania|    1|
  |    United States|            Ireland|  264|
  +-----------------+-------------------+-----+

scala> flights.first.DEST_COUNTRY_NAME // United States

```

## 11.4 트랜스포메이션
- Dataset 트랜스포메이션은 Dataframe과 동일, DataFrame의 모든 트랜스포메이션은 Dataset에서 사용할 수 있다. 

- Dataset은 원형의 JVM 데이터 타입을 다루기 때문에 DataFrame만 사용해서 트랜스포메이션을 수행하는 것보다 좀 더 복잡하고 강력한 데이터 타입으로 트랜스포메이션을 사용할 수 있다.

### 11.4.1 필터링
Flight 클래스를 파라미터로 사용해 불리언값을 반환하는 함수를 만들었다.
- 필터가 정의된 함수를 생성
    - 스파크는 정의된 함수를 사용해 모든 로우를 평가한다. 따라서 매우 많은 자원을 사용하나. 그러므로 단순 필터라면 SQL 표현식을 사용하는 것이 좋다. SQL 표현식을 사용하면 데이터 필터링 비용이 크게 줄어들 뿐만 아니라 다음 처리 과정에서 Dataset으로 데이터를 다룰 수 있다. 정의한 함수를 filter 메서드에 적용해 각 행이 true를 반환하는지 평가하고 데이터셋을 필터링할 수 있다.
- 불리언값은 출발지와 도착지가 동일한지를 나타낸다. 

```
def originIsDestination(flight_row: Flight): Boolean = {
    return flight_row.ORIGIN_COUNTRY_NAME == flight_row.DEST_COUNTRY_NAME
}

# 필터링 함수를 filter 메서드에 적용해 각 행이 true를 반환하는지 평가하고 데이터셋을 필터링 할 수 있다.
flights.fliter(flight_row => originIsDestination(flight_row)).first()

# 결과
# Flight = Flight(United Stated, United States, 348113)

```
- 위 함수는 스파크 코드에서 사용하기 전에 사용자 정의 함수처럼 로컬 머신의 데이터를 대상으로 테스트가 가능하다.

```scala

flights.collct().fliter(flight_row => originIsDestination(flight_row))

# 결과: 함수를 사용했을 때와 동일한 결과가 반환된다.
# Array[Flight] = Array(Flight(United States, United States, 348113))
```


### 11.4.2 매핑
- 매핑은 특정 값을 다른 값으로 매핑해야하는 것을 말한다.

- 매핑 작업
    - Dataset을 다루는 가장 간단한 예제는 로우의 특정 컬럼값을 추출하는 것
    - DataFrame에 매핑 작업을 수행하는 것은 Dataset의 select 메서드를 사용하는 것과 같다. 

```scala
val destinations = flights.map(f => f.DEST_COUNTRY_NAME)

# 최종적으로 String 데이터 타입의 Dataset을 반환한다. 스파크는 결과로 반환할 JVM 데이터 타입을 알고 있기 때문에 컴파일 타임에 데이터 타입의 유효성을 검사할 수 있다.

# 드라이버는 결괏값을 모아 문자열 타입의 배열로 반환한다.

val localDestinations = destinations.take(5)
```

## 11.5 조인
- 조인은 DataFrame에서와 마찬가지로 Dataset에도 동일하게 적용된다. 하지만 Dataset은 joinWith처럼 정교한 메서드를 제공한다. 

- joinWith 메서드는 co-group(RDD 용어)과 거의 유사하며 Dataset 안쪽에 다른 두개의 중첩된 Dataset으로 구성된다. 각 컬럼은 단일 Dataset이므로 Dataset 객체를 컬럼처럼 다룰 수 있다. 그러므로 조인 수행 시 더 많은 정보를 유지할 수 있으며 고급 맵이나 피러처럼 정교하게 데이터를 다룰 수 있다. 

- 예시
    - 가짜 항공운항 메타데이터 데이터셋 생성
    - 최종적으로 로우는 Flight와 FlightMetadata로 이루어진 일종의 키-값 형태의 Dataset을 반환한다. 

```scala

# 가짜 항공운항 메타데이터 데이터셋 생성

scala> case class FlightMetadata(count: BigInt, randomData: BigInt)

scala> val flightsMeta = spark.range(500).map(x => (x, scala.util.Random.nextLong)).withColumnRenamed("_1", "count").withColunRenamed("_2", "roundomData")
    .as(FlightMetadata)

scala> val flight2 = flights.joinWith(flightsMeta, flights.col("count" === fightsMeta.col("count")))


scala> flights2.selectExpr("_1.DEST_COUNTRY_NAME")

scala> flights2.take(2)


```

- 일반 조인도 동작하나, DataFrame 반환으로 JVM 데이터 타입 정보를 모두 잃게 된다. 

```scala
val flights2 = flights.join(flightsMeta. Seq"count")

# 이 정보를 얻으려면 다른 Dataset 정의가 필요
# DataFrame과 Dataset을 조인하는 것은 아무런 문제가 되지 앟으며 최종적으로 동일한 결과를 반환한다.

val flights2 = flights.join(flightsMeta.toDF(), Seq("count"))
```

## 11.6 그룹화와 집계
- 기본 표준을 따른다. groupBy, rollup, cube 메서드를 여전히 사용할 수 있다.
- 하지만 DataFrame을 반환하기 때문에 데이터 타입 정보를 잃게 된다.

- 데이터 타입 정보를 유지하면서 그룹화와 집계 하는 방법이 존재한다. `groupByKey` 메서드는 **Dataset의 특정 키를 기준으로 그룹화하고 형식화된 Dataset을 반환한다.** 하지만 이 함수는 컬럼명 대신 함수를 파라미터로 사용해야 한다. `groupByKey` 메서드의 파라미터로 함수를 사용함으로써 유연성을 얻을 수 있다. 
- 하지만 스파크는 함수와 JVM 데이터 타입을 최적화할 수 없으므로 트레이드오프가 발생한다.
    - 실행 계획으로 이유를 확인 가능

```scala

# 데이터 타입 정보를 잃음
> flighs.groupBy("DEST_COUNTRY_NAME").count()

# 데이터 타입 정보를 잃지 않기 위해서 
> flights.groupByKey(x => x.DEST_COUNTRY_NAME).count()

```

- 실행계획 비교

```scala

> flights.groupBy("DEST_COUNTRY_NAME").count().explain
  == Physical Plan ==
  *HashAggregate(keys=[DEST_COUNTRY_NAME#1308], functions=[count(1)])
  +- Exchange hashpartitioning(DEST_COUNTRY_NAME#1308, 200)
     +- *HashAggregate(keys=[DEST_COUNTRY_NAME#1308], functions=[partial_count(1)])
        +- *FileScan parquet [DEST_COUNTRY_NAME#1308] Batched: tru...


> flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().explain

  == Physical Plan ==
  *HashAggregate(keys=[value#1396], functions=[count(1)])
  +- Exchange hashpartitioning(value#1396, 200)
     +- *HashAggregate(keys=[value#1396], functions=[partial_count(1)])
        +- *Project [value#1396]
           +- AppendColumns <function1>, newInstance(class ...
           [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, ...
              +- *FileScan parquet [D...
```

- groupBykey
    - Dataset의 키를 이용해 그룹화를 수행한 다음 결과를 키-값 형태로 함수에 전달해 원시 객체 형태로 그룹화된 데이터를 다룰 수 있다. 
    - groupByKey 예제처럼 새로운 처리 방법을 생성해 그룹을 축소하는 방법을 정의할 수 있다.

```scala
def grpSum(countryName: String, vlalues: Iterator[Flight]) = {
    values.dropWhile(_.count < 5).map(x => (countryName, x))
}
flights.groupByKey(x => x.DEST_COUNTRY_NAME).flatMapGroups(grpSum).show(5)

def grpSum2(f:Flight):Integer = {
    1
}
flights.groupByKey(x => x.DEST_COUNTRY_NAME).mapValues(grpSum2).count().take(5)

def sum2(left:Flight right:Flight) = {
    Flight(left.DEST_COUNTRY_NAME, null, left.count + right.count)
}

flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(1, r)).take(5)
```
